In [ ]:
from pan3d.xarray.algorithm import vtkXArrayRectilinearSource

In [ ]:
source = vtkXArrayRectilinearSource()
source.load(
    {
        "data_origin": {
            "source": "xarray",
            "id": "eraint_uvz",
        },
        "dataset_config": {
            "arrays": ["u", "v"],
        },
    }
)

In [ ]:
source.computed = {
    "_use_scalars": ["u", "v"],  # optional if empty
    "_use_vectors": [],  # optional if empty
    "vec": "(u * iHat) + (v * jHat)",
    "m2": "u*u + v*v",
}

In [ ]:
print(source())

In [ ]:
import pyvista as pv

plotter = pv.Plotter()
plotter.add_mesh(source, scalars="m2", cmap="coolwarm")
plotter.show()

## Create UI to drive source + viz

In [ ]:
import ipywidgets as widgets


def update_level(change):
    source.slices = {
        **source.slices,
        source.z: change.new,
    }
    plotter.render()


def update_resolution(change):
    v = source.slices
    v[source.x] = [0, -1, change.new]
    v[source.y] = [0, -1, change.new]
    source.slices = v
    plotter.render()


slider_level = widgets.IntSlider(
    value=0,
    min=0,
    max=(source.input[source.z].size - 1),
    step=1,
    description=source.z,
)
slider_level.observe(update_level, names="value")
slider_resolution = widgets.IntSlider(
    value=1, min=1, max=10, step=1, description="resolution"
)
slider_resolution.observe(update_resolution, names="value")
widgets.HBox([slider_level, slider_resolution])